In [ ]:
%pip install tensorflow keras nltk scikit-learn transformers


In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install matplotlib

In [ ]:
pip uninstall transformers -y


In [ ]:
pip install google-generativeai

In [ ]:
pip install nltk

In [ ]:
pip install transformers tensorflow torch scikit-learn matplotlib

In [ ]:
pip cache purge

In [ ]:

pip install keras==2.13.1  transformers==4.33.0

In [ ]:
pip show tensorflow keras transformers

In [ ]:
pip install keras==3.0.5 transformers==4.38.1

In [ ]:
pip install tensorflow-addons


In [ ]:
#Another Version

In [ ]:
pip install sentence-transformers

In [ ]:
#Best Model
import os
import json
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gc
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras import backend as K  # To clear memory
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (Input, Dense, Dropout, LSTM, BatchNormalization, Bidirectional, Layer,
                                     MultiHeadAttention, LayerNormalization, Add , Input)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE

# ---- Step 2: Define Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                   initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                   initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1]) # Output shape is (batch_size, embedding_dim)

# ---- Step 3: Load and Prepare Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]

X_texts = [sample["Context"] for sample in parsed_data]
Y_labels = [sample["Response"] for sample in parsed_data]

# ---- Step 4: Define SBERT Variants to Test ----
sbert_variants = [
    'paraphrase-MiniLM-L12-v2'
]
#',
#,     'all-MiniLM-L6-v2',
embedding_path = "X_emb.npy"
label_path = "Y_encoded.npy"
# ---- Step 5: Encode Labels ----
if os.path.exists(embedding_path) and os.path.exists(label_path):
    print("✅ Loading saved embeddings...")
    X_emb = np.load(embedding_path)
    Y_encoded = np.load(label_path)
    with open("response_encoder.pkl", "rb") as f:
        label_encoder = pickle.load(f)
else:
    print("🔄 Generating SBERT embeddings...")
    sbert = SentenceTransformer(sbert_model_name)
    X_emb = sbert.encode(augmented_inputs, show_progress_bar=True)

    label_encoder = LabelEncoder()
    Y_encoded_labels = label_encoder.fit_transform(Y_labels)
    num_classes_original = len(label_encoder.classes_)
    Y_encoded = to_categorical(Y_encoded_labels, num_classes=num_classes_original)
    np.save(embedding_path, X_emb)
    np.save(label_path, Y_encoded)
    with open("response_encoder.pkl", "wb") as f:
        pickle.dump(label_encoder, f)


# ---- Step 6: Save Label Encoder ----
with open("response_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# ---- Step 7: Remove Rare Classes (only 1 sample) ----
class_counts = Counter(np.argmax(Y_encoded, axis=1))
valid_classes = {cls for cls, count in class_counts.items() if count > 1}
valid_indices = [i for i, label in enumerate(np.argmax(Y_encoded, axis=1)) if label in valid_classes]

X_texts_filtered = [X_texts[i] for i in valid_indices]
# Re-extract filtered string labels
Y_labels_filtered = [Y_labels[i] for i in valid_indices]

# Re-encode using a new LabelEncoder for filtered classes
label_encoder = LabelEncoder()
Y_filtered_int = label_encoder.fit_transform(Y_labels_filtered)

# Save new encoder
with open("response_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Now one-hot encode with correct number of classes
current_num_classes = len(label_encoder.classes_)
Y_filtered = to_categorical(Y_filtered_int, num_classes=current_num_classes)

# ---- Step 8: Prepare Embeddings and Split Data ----
sbert_model_name = sbert_variants[0] # Using the first variant for this non-function block
print(f"\n🔄 Preparing embeddings with SBERT model: {sbert_model_name}")
sbert = SentenceTransformer(sbert_model_name)
X_emb = sbert.encode(X_texts_filtered)
Y_filtered_encoded_labels_step8 = np.argmax(Y_filtered, axis=1) # Get integer labels after filtering

# Resample using SMOTE + TomekLinks if safe
if min_samples_per_class < 2:
    print("⚠️ SMOTE skipped due to classes with <2 samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered_encoded_labels_step8
else:
    smote_k = min(5, min_samples_per_class - 1)
    smote_k = max(smote_k, 1)
    print(f"✅ Applying SMOTE with k={smote_k} and TomekLinks...")
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote_encoded = smote.fit_resample(X_emb, Y_filtered_encoded_labels_step8)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote_encoded)

unique_resampled_labels = np.unique(Y_resampled_encoded)
current_num_classes = len(unique_resampled_labels)

# --- FIX: Ensure Y_resampled_encoded values are within the valid range ---
Y_resampled_encoded = np.clip(Y_resampled_encoded, 0, current_num_classes - 1)

Y_resampled = to_categorical(Y_resampled_encoded, num_classes=current_num_classes)
print(f"📊 Resampled Class Distribution: {Counter(np.argmax(Y_resampled, axis=1))}")
print(f"🔢 Number of classes after resampling: {current_num_classes}")

# Train-Test Split (75% train, 25% test)
print(f"➡️ Before train_test_split: test_size = 0.25")
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled,
    test_size=0.25,  # 25% for the test set
    random_state=42,
    stratify=np.argmax(Y_resampled, axis=1)
)
print(f"✅ After train_test_split: Training size = {len(X_train)}, Testing size = {len(X_test)}")

# Reshape input for LSTM
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(Y_train_int),
    y=Y_train_int
)
class_weight_dict = dict(enumerate(class_weights))

print(f"📏 Training data size: {len(X_train)}")
print(f"📏 Testing data size: {len(X_test)}")
# ---- Step 9: Build Model ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout_rate=0.3):
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention_output = Dropout(dropout_rate)(attention_output)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention_output]))

        ff_output = Dense(ff_dim, activation="relu")(out1)
        ff_output = Dense(inputs.shape[-1])(ff_output)
        ff_output = Dropout(dropout_rate)(ff_output)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff_output]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)

    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(num_classes, activation="softmax")(x)

    lr_schedule = ExponentialDecay(
        initial_learning_rate=0.0005,
        decay_steps=1100,
        decay_rate=0.95
    )

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=AdamW(learning_rate=lr_schedule), metrics=["accuracy"])
    return model

# Build the model
model = build_model(input_shape=X_train.shape[1:], num_classes=current_num_classes)

# ---- Step 10: Train Model ----
print(f"\n🚀 Training model using: {sbert_model_name}")

# 🧹 Clear session
K.clear_session()
gc.collect()

early_stop = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, verbose=1, min_lr=1e-6)

history = model.fit(
    X_train, Y_train_int,  # Use integer labels for training
    validation_data=(X_test, Y_test_int),  # Use integer labels for validation
    epochs=20,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=[early_stop],
    verbose=1
)

# ---- Step 11: Evaluate and Save Model ----
test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)  # Use integer labels for evaluation
print(f"✅ Test Accuracy with {sbert_model_name}: {test_acc * 100:.2f}%")
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import json
import random
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

# ---- Custom Attention Layer (required for model loading) ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

# ---- File Paths ----
MODEL_PATH = "optimized_lstm_model.keras"
LABEL_ENCODER_PATH = "response_encoder.pkl"
QUESTIONNAIRE_PATH = "generated_questionnaire.json"
MOOD_KEYWORDS_PATH = "mood_keywords.json"
SBERT_MODEL_NAME = "paraphrase-MiniLM-L12-v2"

# ---- Load Assets ----
model = load_model(MODEL_PATH, custom_objects={"AttentionLayer": AttentionLayer})

with open(LABEL_ENCODER_PATH, "rb") as f:
    label_encoder = pickle.load(f)

with open(QUESTIONNAIRE_PATH, "r") as f:
    questionnaire = json.load(f)

with open(MOOD_KEYWORDS_PATH, "r") as f:
    mood_keywords = json.load(f)

sbert = SentenceTransformer(SBERT_MODEL_NAME)

# ---- Helper Functions ----
def augment_input_with_mood_keywords(user_input):
    tokens = []
    for mood, keywords in mood_keywords.items():
        if any(word.lower() in user_input.lower() for word in keywords):
            tokens.append(f"intent:{mood.lower()}")
    return user_input + " " + " ".join(tokens)

def predict_response(user_input):
    """Return full response from the model"""
    augmented = augment_input_with_mood_keywords(user_input)
    embedding = sbert.encode([augmented])
    embedding = np.expand_dims(embedding, axis=1)
    prediction = model.predict(embedding, verbose=0)
    pred_index = np.argmax(prediction, axis=1)[0]
    return label_encoder.inverse_transform([pred_index])[0]

def detect_mood_from_response(response_text):
    """Infer mood category from keywords in the model response"""
    for mood, keywords in mood_keywords.items():
        if any(word.lower() in response_text.lower() for word in keywords):
            return mood
    return "Unknown"

def get_question_for_mood(mood):
    """Select a question based on detected mood"""
    for category, questions in questionnaire.items():
        if mood.lower() in category.lower():
            return random.choice(questions)
    return "Sorry, I couldn't find a relevant question for you."

# ---- Chat Loop ----
def chatbot():
    print("🧠 Mental Health Bot: Hello! I'm here to support you. Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("🧠 Bot: Take care and stay safe! 💙")
            break

        full_response = predict_response(user_input)
        detected_mood = detect_mood_from_response(full_response)
        question = get_question_for_mood(detected_mood)

        print(f"\n🧠 Bot (Mood Detected: {detected_mood})")
        print(f"→ Response: {full_response}")
        print(f"→ Follow-up Question: {question}\n")

# ---- Run ----
if __name__ == "__main__":
    chatbot()


In [ ]:
import os
import json
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional, BatchNormalization, Layer, Add, LayerNormalization, MultiHeadAttention
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]
contexts = [sample["Context"] for sample in parsed_data]

# ---- Load Mood Keywords ----
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

def detect_mood(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

X_texts, Y_labels = [], []
for context in contexts:
    mood = detect_mood(context)
    if mood:
        X_texts.append(context)
        Y_labels.append(mood)

# ---- Encode and Filter Labels ----
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_labels)
with open("mood_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

label_counts = Counter(Y_encoded)
valid_labels = {label for label, count in label_counts.items() if count > 1}
filtered = [(x, y) for x, y in zip(X_texts, Y_encoded) if y in valid_labels]
X_texts_filtered = [x for x, _ in filtered]
Y_filtered = [y for _, y in filtered]

# ---- SBERT Embedding ----
sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
print("🔄 Generating SBERT embeddings...")
X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)

# ---- Resampling ----
min_samples = min(Counter(Y_filtered).values())
if min_samples < 2:
    print("⚠️ SMOTE skipped due to insufficient samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered
else:
    smote_k = max(1, min(5, min_samples - 1))
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote = smote.fit_resample(X_emb, Y_filtered)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote)

Y_resampled = to_categorical(Y_resampled_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled, test_size=0.25, stratify=np.argmax(Y_resampled, axis=1), random_state=42
)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

class_weights = compute_class_weight("balanced", classes=np.unique(Y_train_int), y=Y_train_int)
class_weight_dict = dict(enumerate(class_weights))

# ---- Model Builder ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout=0.3):
        attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention = Dropout(dropout)(attention)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention]))
        ff = Dense(ff_dim, activation='relu')(out1)
        ff = Dense(inputs.shape[-1])(ff)
        ff = Dropout(dropout)(ff)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ---- Train Model ----
gc.collect()
model = build_model(input_shape=X_train.shape[1:], num_classes=Y_resampled.shape[1])
early_stop = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, min_lr=1e-6, verbose=1)

In [ ]:
print("🚀 Training model...")
history = model.fit(
    X_train, Y_train_int,
    validation_data=(X_test, Y_test_int),
    epochs=20,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")
model.save("mood_classifier_model.keras", save_format="keras")
print("💾 Model saved as mood_classifier_model.keras")

# ---- Plot Results ----
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.title('Accuracy'); plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('Loss'); plt.legend()
plt.tight_layout()
plt.show()

# ---- F1 Score ----
Y_pred_probs = model.predict(X_test, verbose=0)
Y_pred = np.argmax(Y_pred_probs, axis=1)
f1 = f1_score(Y_test_int, Y_pred, average="weighted")
print(f"🎯 Weighted F1 Score: {f1 * 100:.2f}%")

In [ ]:
import os
import json
import pickle
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Model ----
try:
    model = load_model("mood_classifier_model.keras", custom_objects={"AttentionLayer": AttentionLayer})
    print("✅ Model loaded cleanly.")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    exit()

# ---- Load SBERT and Resources ----
sbert = SentenceTransformer("paraphrase-MiniLM-L12-v2")

with open("mood_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

with open("extracted_questions.json", "r", encoding="utf-8") as f:
    question_data = json.load(f)

# ---- Logging ----
LOG_FILE = "chat_log.json"

def log_user_query(user_input, mood, question):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_mood": mood,
        "response_question": question
    }
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r", encoding="utf-8") as f:
            logs = json.load(f)
    else:
        logs = []
    logs.append(entry)
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        json.dump(logs, f, indent=2)
    print("📝 Query logged.")

# ---- Mood Detection ----
def detect_mood_keywords(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

def predict_mood(text):
    mood = detect_mood_keywords(text)
    if mood:
        print(f"📌 Detected mood from keywords: {mood}")
        return mood

    embedding = sbert.encode([text])
    embedding = np.expand_dims(embedding, axis=1)  # Shape: (1, 1, 384)
    prediction = model.predict(embedding, verbose=0)
    mood_index = np.argmax(prediction)
    mood_label = label_encoder.inverse_transform([mood_index])[0]
    print(f"🤖 Predicted mood from model: {mood_label}")
    return mood_label

# ---- Fetch Question ----
def fetch_question(mood):
    mood_sections = question_data.get(mood, [])
    all_questions = []

    for section in mood_sections:
        general = section.get("General", {})
        questions = general.get("questions", [])
        if isinstance(questions, list):
            all_questions.extend(questions)

    if not all_questions:
        return "I'm here to support you. Can you tell me more about how you're feeling?"
    return np.random.choice(all_questions)

# ---- Chatbot Main ----
def chatbot():
    print("🧠 ChatBot is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Take care. I'm here whenever you need support.")
            break

        mood = predict_mood(user_input)
        question = fetch_question(mood)
        print(f"MoodBot ({mood}): {question}\n")
        log_user_query(user_input, mood, question)

# ---- Run ----
if __name__ == "__main__":
    chatbot()


In [ ]:
#This is Another version OF the code , 
import json
import nltk
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

# Load your clustered short mood dataset
with open("short_mood_dataset.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]

# Group context texts by mood
mood_to_contexts = defaultdict(list)
for sample in data:
    mood = sample["Mood"]
    context = sample["Context"]
    if mood and context:
        mood_to_contexts[mood].append(context.lower())

# Extract top keywords for each mood label
mood_keywords = {}
for mood, contexts in mood_to_contexts.items():
    all_text = " ".join(contexts)
    words = word_tokenize(all_text)
    cleaned_words = [
        w.lower() for w in words
        if w.isalpha() and w.lower() not in stop_words
    ]
    word_freq = Counter(cleaned_words)
    top_keywords = [word for word, _ in word_freq.most_common(15)]
    if top_keywords:
        mood_keywords[mood] = top_keywords

# Save as mood_keywords.json
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(mood_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords.json generated with {len(mood_keywords)} mood labels.")


In [ ]:
import os
import json
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gc
import nltk
import tensorflow as tf
from tensorflow.keras import backend as K  # To clear memory
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (Input, Dense, Dropout, LSTM, BatchNormalization, Bidirectional, Layer,
                                     MultiHeadAttention, LayerNormalization, Add)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GaussianNoise
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# ---- Step 2: Define Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                   initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                   initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1]) # Output shape is (batch_size, embedding_dim)
    
# ---- Step 3: Load and Prepare Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]

X_texts = [sample["Context"] for sample in parsed_data]
Y_labels = [sample["Response"] for sample in parsed_data]

# Group "Context" texts by "Response" label
label_to_contexts = defaultdict(list)
for item in parsed_data:
    context = item.get("Context", "")
    response = item.get("Response", "")
    if context and response:
        label_to_contexts[response].append(context.lower())

# Extract keywords for each label
mood_keywords = {}
for label, contexts in label_to_contexts.items():
    all_text = " ".join(contexts)
    words = word_tokenize(all_text)
    cleaned_words = [
        w.lower() for w in words
        if w.isalpha() and w.lower() not in stop_words
    ]
    word_freq = Counter(cleaned_words)
    top_keywords = [word for word, _ in word_freq.most_common(20)]  # More keywords for broader intent detection
    mood_keywords[label] = top_keywords

# Save to mood_keywords.json
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(mood_keywords, f, indent=2, ensure_ascii=False)

print("✅ mood_keywords.json generated and saved successfully!")

# ---- Step 4: Define SBERT Variants to Test ----
sbert_variants = [
    'paraphrase-MiniLM-L12-v2'
]
#',
#,     'all-MiniLM-L6-v2',
# ---- Step 5: Encode Labels ----
label_encoder = LabelEncoder()
Y_encoded_labels = label_encoder.fit_transform(Y_labels)
num_classes_original = len(label_encoder.classes_)
Y_encoded = to_categorical(Y_encoded_labels, num_classes=num_classes_original)

# ---- Step 6: Save Label Encoder ----
with open("response_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# ---- Step 7: Remove Rare Classes (only 1 sample) ----
class_counts = Counter(np.argmax(Y_encoded, axis=1))
valid_classes = {cls for cls, count in class_counts.items() if count > 1}
valid_indices = [i for i, label in enumerate(np.argmax(Y_encoded, axis=1)) if label in valid_classes]

X_texts_filtered = [X_texts[i] for i in valid_indices]
Y_filtered = Y_encoded[valid_indices]
Y_filtered_encoded_labels = np.argmax(Y_filtered, axis=1) # Get integer labels after filtering

filtered_class_counts = Counter(Y_filtered_encoded_labels)
min_samples_per_class = min(filtered_class_counts.values())
print(f"📊 Filtered Class Distribution: {filtered_class_counts}")

# ---- Step 8: Prepare Embeddings and Split Data ----
sbert_model_name = sbert_variants[0] # Using the first variant for this non-function block
print(f"\n🔄 Preparing embeddings with SBERT model: {sbert_model_name}")
sbert = SentenceTransformer(sbert_model_name)
X_emb = sbert.encode(X_texts_filtered)
Y_filtered_encoded_labels_step8 = np.argmax(Y_filtered, axis=1) # Get integer labels after filtering

# Resample using SMOTE + TomekLinks if safe
if min_samples_per_class < 2:
    print("⚠️ SMOTE skipped due to classes with <2 samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered_encoded_labels_step8
else:
    smote_k = min(5, min_samples_per_class - 1)
    smote_k = max(smote_k, 1)
    print(f"✅ Applying SMOTE with k={smote_k} and TomekLinks...")
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote_encoded = smote.fit_resample(X_emb, Y_filtered_encoded_labels_step8)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote_encoded)

unique_resampled_labels = np.unique(Y_resampled_encoded)
current_num_classes = len(unique_resampled_labels)

# --- FIX: Ensure Y_resampled_encoded values are within the valid range ---
Y_resampled_encoded = np.clip(Y_resampled_encoded, 0, current_num_classes - 1)

Y_resampled = to_categorical(Y_resampled_encoded, num_classes=current_num_classes)
print(f"📊 Resampled Class Distribution: {Counter(np.argmax(Y_resampled, axis=1))}")
print(f"🔢 Number of classes after resampling: {current_num_classes}")

# Train-Test Split (75% train, 25% test)
print(f"➡️ Before train_test_split: test_size = 0.25")
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled,
    test_size=0.25,  # 25% for the test set
    random_state=42,
    stratify=np.argmax(Y_resampled, axis=1)
)
print(f"✅ After train_test_split: Training size = {len(X_train)}, Testing size = {len(X_test)}")

# Reshape input for LSTM
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(Y_train_int),
    y=Y_train_int
)
class_weight_dict = dict(enumerate(class_weights))

print(f"📏 Training data size: {len(X_train)}")
print(f"📏 Testing data size: {len(X_test)}")
# ---- Step 9: Build Model ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout_rate=0.3):
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention_output = Dropout(dropout_rate)(attention_output)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention_output]))

        ff_output = Dense(ff_dim, activation="relu")(out1)
        ff_output = Dense(inputs.shape[-1])(ff_output)
        ff_output = Dropout(dropout_rate)(ff_output)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff_output]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)

    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)

    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(num_classes, activation="softmax")(x)

    lr_schedule = ExponentialDecay(
        initial_learning_rate=0.002,
        decay_steps=1200,
        decay_rate=0.97
    )

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Build the model
model = build_model(input_shape=X_train.shape[1:], num_classes=current_num_classes)

# ---- Step 10: Train Model ----
print(f"\n🚀 Training model using: {sbert_model_name}")

# 🧹 Clear session
K.clear_session()
gc.collect()

early_stop = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, verbose=1, min_lr=1e-6)

history = model.fit(
    X_train, Y_train_int,  # Use integer labels for training
    validation_data=(X_test, Y_test_int),  # Use integer labels for validation
    epochs=30,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr, early_stop],
    verbose=1
)

# ---- Step 11: Evaluate and Save Model ----
test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)  # Use integer labels for evaluation
Y_pred_probs = model.predict(X_test, verbose=0)
Y_pred = np.argmax(Y_pred_probs, axis=1)
print(f"✅ Test Accuracy with {sbert_model_name}: {test_acc * 100:.2f}%")
model_filename = f"model_{sbert_model_name.replace('-', '_')}.keras"
if os.path.exists(model_filename):
    os.remove(model_filename)
model.save(model_filename)
print(f"💾 Model saved as {model_filename}")

# --- Optional: Plot training history ---
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

# ---- Step 12: Save Model as Final ----
final_model_name = "optimized_lstm_model.keras"
if os.path.exists(final_model_name):
    try:
        previous_model = load_model(final_model_name, custom_objects={"AttentionLayer": AttentionLayer})
        _, prev_acc = previous_model.evaluate(X_test, Y_test_int, verbose=1)
        if test_acc > prev_acc:
            os.replace(model_filename, final_model_name)
            print(f"✅ Final model updated → Accuracy improved from {prev_acc*100:.2f}% → {test_acc*100:.2f}%")
        else:
            print(f"⚠️ Final model not updated (existing model has higher accuracy: {prev_acc*100:.2f}%)")
    except Exception as e:
        print(f"⚠️ Could not load existing model. Replacing with new one. Error: {e}")
        os.replace(model_filename, final_model_name)
else:
    os.replace(model_filename, final_model_name)
    print(f"✅ Final model saved as {final_model_name}")

f1 = f1_score(Y_test_int, Y_pred, average="weighted")
print(f"✅ Test Accuracy with {sbert_model_name}: {test_acc * 100:.2f}%")
print(f"🎯 Weighted F1 Score: {f1 * 100:.2f}%")


In [ ]:
import json
import nltk
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# NLTK setup
nltk.download("punkt")
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Load combined_dataset.json
with open("combined_dataset.json", "r", encoding="utf-8") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]

# Load previously generated mood keywords (from context analysis)
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    context_mood_keywords = json.load(f)

# Helper: Detect mood from context using context-based keywords
def detect_mood(text, mood_keywords):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

# Step 1: Collect grouped text by mood
mood_to_contexts = defaultdict(list)
mood_to_responses = defaultdict(list)

for sample in parsed_data:
    context = sample.get("Context", "").lower()
    response = sample.get("Response", "").lower()
    detected_mood = detect_mood(context, context_mood_keywords)
    if detected_mood:
        if context:  mood_to_contexts[detected_mood].append(context)
        if response: mood_to_responses[detected_mood].append(response)

# Step 2: Extract keywords from both context & response
def extract_keywords(texts, top_n=15):
    all_text = " ".join(texts)
    tokens = word_tokenize(all_text)
    words = [w for w in tokens if w.isalpha() and w not in stop_words]
    freq = Counter(words)
    return [word for word, _ in freq.most_common(top_n)]

combined_keywords = {}

all_moods = set(mood_to_contexts) | set(mood_to_responses)

for mood in all_moods:
    context_keywords = extract_keywords(mood_to_contexts[mood]) if mood in mood_to_contexts else []
    response_keywords = extract_keywords(mood_to_responses[mood]) if mood in mood_to_responses else []
    combined_keywords[mood] = list(set(context_keywords + response_keywords))

# Step 3: Save combined mood keyword file
with open("mood_keywords_combined.json", "w", encoding="utf-8") as f:
    json.dump(combined_keywords, f, indent=2, ensure_ascii=False)

print("✅ mood_keywords_combined.json generated successfully.")


In [ ]:
import os, json, pickle, numpy as np
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from tensorflow.nn import softmax
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from datetime import datetime

# ---- Custom Attention Layer for loading the model ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super().build(input_shape)
    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Paths and Setup ----
model_path = "mood_model_combined.keras"
encoder_path = "mood_encoder.pkl"
sbert_model_name = "paraphrase-MiniLM-L12-v2"
log_path = "chat_log.json"

# ---- Load Model and Encoder ----
print("🔄 Loading mood classifier model...")
model = load_model(model_path, custom_objects={"AttentionLayer": AttentionLayer})
with open(encoder_path, "rb") as f:
    label_encoder = pickle.load(f)
sbert = SentenceTransformer(sbert_model_name)

# ---- Fallback Prompts ----
fallback_responses = [
    "I'm not sure I understood. Could you tell me more about how you feel?",
    "That sounds important. Could you elaborate a bit?",
    "Thanks for sharing. Could you explain it a little more?"
]

# ---- Ensure Chat Log Exists ----
if not os.path.exists(log_path):
    with open(log_path, "w") as f:
        json.dump([], f)

# ---- Prediction Function ----
def predict_mood(text):
    emb = sbert.encode([text])
    emb = np.expand_dims(emb, axis=1)  # Reshape for model input
    probs = softmax(model.predict(emb, verbose=0)[0]).numpy()
    top_idx = np.argmax(probs)
    top_label = label_encoder.inverse_transform([top_idx])[0]
    confidence = probs[top_idx]
    return top_label, confidence, probs

# ---- Log Interaction ----
def log_interaction(user_input, mood, confidence):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "input": user_input,
        "predicted_mood": mood,
        "confidence": float(confidence)
    }
    with open(log_path, "r+") as f:
        logs = json.load(f)
        logs.append(entry)
        f.seek(0)
        json.dump(logs, f, indent=2)

# ---- Chat Loop ----
print("💬 MoodBot is ready. Type 'exit' to quit.")
while True:
    user_input = input("🧠 You: ").strip()
    if user_input.lower() in {"exit", "quit"}:
        print("👋 Goodbye! Take care.")
        break

    predicted_mood, conf, prob = predict_mood(user_input)
    if conf < 0.45:
        response = np.random.choice(fallback_responses)
        print(f"🤖 Bot: {response}")
    else:
        print(f"🤖 Detected Mood: {predicted_mood} ({conf:.2f} confidence)")
        print(f"📩 Prompt: How does this mood affect your day?")

    log_interaction(user_input, predicted_mood, conf)
